In [1]:
import pylab
import matplotlib
import numpy as np
import pandas as pd
from skimage.util import img_as_float
from skimage.io import imread

Загрузите картинку parrots.jpg. Преобразуйте изображение, приведя все значения в интервал от 0 до 1. Для этого можно воспользоваться функцией img_as_float из модуля skimage. Обратите внимание на этот шаг, так как при работе с исходным изображением вы получите некорректный результат.

In [2]:
image = img_as_float(imread('parrots.jpg'))
w, h, d = image.shape

In [3]:
for mass in image:
    for arr in mass:
        for num in arr:
            if num > 1:
                print 'IT\'S a TRAP!'

Создайте матрицу объекты-признаки: характеризуйте каждый пиксель тремя координатами - значениями интенсивности в пространстве RGB.

In [4]:
pixels = pd.DataFrame(np.reshape(image, (w*h, d)), columns=['R', 'G', 'B'])
pixels.head()

,R,G,B
0,0.015686,0.494118,0.019608
1,0.007843,0.494118,0.007843
2,0.007843,0.494118,0.007843
3,0.007843,0.494118,0.007843
4,0.007843,0.501961,0.011765


Запустите алгоритм K-Means с параметрами init='k-means++' и random_state=241. После выделения кластеров все пиксели, отнесенные в один кластер, попробуйте заполнить двумя способами: медианным и средним цветом по кластеру.

In [5]:
from sklearn.cluster import KMeans
%matplotlib inline

In [9]:
def search(pixels,n):
    X = pixels.copy()
    kmean = KMeans(n_clusters=n,init='k-means++', random_state=241).fit(X)
    X['Label']= kmean.labels_
    
    medians, means = X.groupby('Label').median().values, X.groupby('Label').mean().values
    
    medians_pixels, means_pixels = [medians[c] for c in X['Label'].values], [means[c] for c in X['Label'].values]
    
    medians_img, means_img = np.reshape(medians_pixels, (w,h,d)), np.reshape(means_pixels, (w,h,d))
      
    return means_img, medians_img

In [10]:
from math import log10
def psnr(img1,img2):
    mse = np.mean((img1-img2)**2)
    return 10 * log10(1/mse)

Найдите минимальное количество кластеров, при котором значение PSNR выше 20 (можно рассмотреть не более 20 кластеров, но не забудьте рассмотреть оба способа заполнения пикселей одного кластера). Это число и будет ответом в данной задаче.


In [12]:
from skimage.measure import compare_psnr
for i in range(1, 21):
    tmp = search(pixels,i)
    print i, psnr(image,tmp[0]), psnr(image,tmp[1])
    #print i, compare_psnr(image, tmp[0]), compare_psnr(image, tmp[1])
    if psnr(image,tmp[0])>20 or psnr(image,tmp[1])>20:
        break

1 9.81883554477 9.42784046051
2 10.1392421177 9.77739410817
3 10.901642103 10.6534030919
4 11.7295100644 11.2172086545
5 12.61312503 12.1289572256
6 13.4353763973 12.9531724669
7 13.3127130777 12.521697857
8 14.3672321069 13.5632779116
9 15.3673877994 14.5485305192
10 16.4852751635 15.9323733011
11 17.2886709935 16.6569058334
12 18.6515723376 18.3852771505
13 19.4478611145 19.1929260722
14 20.1329059648 19.8638836864
